# Schema-on-read Using Spark

Import `pyspark.sql.types.xxx` for the data types required by schema-on-read.

In [1]:
import os
import sys

from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, DateType, IntegerType

In [2]:
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [3]:
spark = SparkSession.builder.appName("Schema on Read").getOrCreate()

Read from data source

In [4]:
data_file_no_header = "data/schema_on_read_no_header.csv"
df_spark_no_header = spark.read.csv(data_file_no_header, header=False)

data_file_with_header = "data/schema_on_read_with_header.csv"
df_spark_with_header = spark.read.csv(data_file_with_header, header=True)

Inferred schema from CSV header. Notice that all is string.

In [5]:
df_spark_with_header.printSchema()

root
 |-- brand: string (nullable = true)
 |-- model: string (nullable = true)
 |-- release_date: string (nullable = true)
 |-- price: string (nullable = true)



In [6]:
df_spark_with_header.show(5)

+----------+------+------------+-----+
|     brand| model|release_date|price|
+----------+------+------------+-----+
|      Audi|    A8|  2019-11-28|18692|
|     Volvo|   C70|  2020-05-01|23396|
|      Ford|  F450|  2020-02-14|23811|
|Mitsubishi|Tredia|  2019-03-06|17035|
|       Kia|   Rio|  2020-04-04|16921|
+----------+------+------------+-----+
only showing top 5 rows



No header, so all rows are treated as data. The inferred schema has no column name.

In [7]:
df_spark_no_header.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)



In [8]:
df_spark_no_header.show(5)

+----------+-----------+----------+-----+
|       _c0|        _c1|       _c2|  _c3|
+----------+-----------+----------+-----+
|       GMC|     Sonoma|2020-03-14|21062|
|Land Rover|Range Rover|2019-04-18|16563|
|     Honda|    Insight|2021-01-01|24766|
|      Ford|       F350|2019-01-22|20303|
|    Subaru|        SVX|2019-04-21|24681|
+----------+-----------+----------+-----+
only showing top 5 rows



Define our own schema explicitly.

In [9]:
spark_schema = StructType([
    StructField("f_brand", StringType()),
    StructField("f_model", StringType()),
    StructField("f_release_date", DateType()),
    StructField("f_price", IntegerType())
])

Read csv using explicit_schema

In [10]:
df_spark_with_header_explicit_schema = spark.read.csv(
    data_file_with_header, header=True, schema=spark_schema, dateFormat="yyyy-mm-dd")

df_spark_no_header_explicit_schema = spark.read.csv(
    data_file_no_header, header=False, schema=spark_schema, dateFormat="yyyy-mm-dd")

When using explicit schema. Notice the data type.

In [11]:
df_spark_with_header_explicit_schema.printSchema()

root
 |-- f_brand: string (nullable = true)
 |-- f_model: string (nullable = true)
 |-- f_release_date: date (nullable = true)
 |-- f_price: integer (nullable = true)



In [12]:
df_spark_no_header_explicit_schema.printSchema()

root
 |-- f_brand: string (nullable = true)
 |-- f_model: string (nullable = true)
 |-- f_release_date: date (nullable = true)
 |-- f_price: integer (nullable = true)



Since the data type is correct, we can process them accordingly. For example, add one day to release date.

In [13]:
df_spark_with_header_explicit_schema.createOrReplaceTempView("cars_explicit_schema")

In [15]:
spark.sql("SELECT f_brand, f_model, f_release_date + 1 "\
          "  FROM cars_explicit_schema ")\
.show()

+----------+--------------+---------------------------+
|   f_brand|       f_model|date_add(f_release_date, 1)|
+----------+--------------+---------------------------+
|      Audi|            A8|                 2019-01-29|
|     Volvo|           C70|                 2020-01-02|
|      Ford|          F450|                 2020-01-15|
|Mitsubishi|        Tredia|                 2019-01-07|
|       Kia|           Rio|                 2020-01-05|
|     Buick|   Park Avenue|                 2020-01-11|
|       Kia|           Rio|                 2019-01-15|
|      Ford|          Edge|                 2020-01-25|
| Chevrolet|        Camaro|                 2021-01-14|
|      Audi|4000CS Quattro|                 2020-01-22|
|Mitsubishi|      Diamante|                 2019-01-08|
|     Acura|            CL|                 2019-01-17|
|     Dodge|        Dakota|                 2019-01-25|
|     Buick|       Century|                 2019-01-31|
|Mitsubishi|        Pajero|                 2020

Notice the difference, when we process the data with inferred schema, where date considered as string.

In [16]:
df_spark_with_header.createOrReplaceTempView("cars_inferred_schema")
df_spark_with_header.printSchema()

root
 |-- brand: string (nullable = true)
 |-- model: string (nullable = true)
 |-- release_date: string (nullable = true)
 |-- price: string (nullable = true)



In [23]:
spark.sql("SELECT brand, model, release_date + 1"\
          "  FROM cars_inferred_schema ")\
.show()

+----------+--------------+------------------+
|     brand|         model|(release_date + 1)|
+----------+--------------+------------------+
|      Audi|            A8|              null|
|     Volvo|           C70|              null|
|      Ford|          F450|              null|
|Mitsubishi|        Tredia|              null|
|       Kia|           Rio|              null|
|     Buick|   Park Avenue|              null|
|       Kia|           Rio|              null|
|      Ford|          Edge|              null|
| Chevrolet|        Camaro|              null|
|      Audi|4000CS Quattro|              null|
|Mitsubishi|      Diamante|              null|
|     Acura|            CL|              null|
|     Dodge|        Dakota|              null|
|     Buick|       Century|              null|
|Mitsubishi|        Pajero|              null|
|Oldsmobile|    Silhouette|              null|
|     Mazda|      B-Series|              null|
|      Ford|      F-Series|              null|
|  Infiniti| 

Inferred schema sometimes gives expected value, but better not to rely on them unless you're sure. See the price column.

In [19]:
spark.sql("SELECT brand, model, release_date + 1, price / 2 "\
          "  FROM cars_inferred_schema")\
.show()

+----------+--------------+------------------+-----------+
|     brand|         model|(release_date + 1)|(price / 2)|
+----------+--------------+------------------+-----------+
|      Audi|            A8|              null|     9346.0|
|     Volvo|           C70|              null|    11698.0|
|      Ford|          F450|              null|    11905.5|
|Mitsubishi|        Tredia|              null|     8517.5|
|       Kia|           Rio|              null|     8460.5|
|     Buick|   Park Avenue|              null|    10139.0|
|       Kia|           Rio|              null|     9813.5|
|      Ford|          Edge|              null|     8326.5|
| Chevrolet|        Camaro|              null|     8688.0|
|      Audi|4000CS Quattro|              null|     8247.0|
|Mitsubishi|      Diamante|              null|    12144.0|
|     Acura|            CL|              null|     9498.0|
|     Dodge|        Dakota|              null|    10817.0|
|     Buick|       Century|              null|    10720.

Same result with explicit schema

In [20]:
spark.sql("SELECT f_brand, f_model, f_price / 2 "\
          "  FROM cars_explicit_schema")\
.show()

+----------+--------------+-------------+
|   f_brand|       f_model|(f_price / 2)|
+----------+--------------+-------------+
|      Audi|            A8|       9346.0|
|     Volvo|           C70|      11698.0|
|      Ford|          F450|      11905.5|
|Mitsubishi|        Tredia|       8517.5|
|       Kia|           Rio|       8460.5|
|     Buick|   Park Avenue|      10139.0|
|       Kia|           Rio|       9813.5|
|      Ford|          Edge|       8326.5|
| Chevrolet|        Camaro|       8688.0|
|      Audi|4000CS Quattro|       8247.0|
|Mitsubishi|      Diamante|      12144.0|
|     Acura|            CL|       9498.0|
|     Dodge|        Dakota|      10817.0|
|     Buick|       Century|      10720.5|
|Mitsubishi|        Pajero|       8336.0|
|Oldsmobile|    Silhouette|       8932.0|
|     Mazda|      B-Series|      10013.5|
|      Ford|      F-Series|      11703.5|
|  Infiniti|            QX|      10554.0|
| Chevrolet|  Express 2500|      11678.5|
+----------+--------------+-------